In [1]:
import requests
from bs4 import BeautifulSoup

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
URL_BASE = 'https://www.cfcprodutivo.com.br'
URL_BUSCA = URL_BASE + '/framework/admin/ajax/'
URL_PERFIL = URL_BASE + '/framework/admin/editar/alunos/'

In [3]:
s = requests.session()
URL_LOGIN = 'https://cfcprodutivo.com.br/framework/admin/ajax/login.php'

payload = {'login': 'cfcadm',
            'senha': 'abc123'}

login = s.request('POST', URL_LOGIN, data=payload)

In [4]:

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'referer': 'https://www.cfcprodutivo.com.br/framework/admin/listar/alunos',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

params = {
    'iDisplayStart': '0',
    'iDisplayLength': '100',
    'iSortCol_0': '1',
    'sSortDir_0': 'desc',
}

response = s.get(
    'https://www.cfcprodutivo.com.br/framework/admin/ajax/ajaxListaClientesAlunos.php',
    params=params,
    headers=headers,
)

if 'douglas-ferreira-alves' in response.text:
    print('ACHOU')

ACHOU


In [5]:
soap = BeautifulSoup(response.content, 'html.parser')


In [6]:
soap = BeautifulSoup(response.content, 'html.parser')
links = soap.findAll('a')

for link in links:
    url_form = link['href'].replace('\\', '').replace('"', '')
    response_page = s.request("GET", URL_BASE + url_form)
    break
    

In [7]:
import re 
id = int(re.search(r'\d+', response_page.url).group())

URL_FICHA = URL_BUSCA + 'containerTabClientesAlunosVerFicha.php?'

In [8]:

response_perfil = s.request("GET", URL_PERFIL + str(id))
soap = BeautifulSoup(response_perfil.content, 'html.parser')
infos = dict()

# for linha in soap.findAll("input"):
#     try: key = linha['name']
#     except: continue

#     try:value = linha['value']
#     except:value = ''

    
dados_bancarios = soap.find('textarea', {'id': 'dados_bancarios'}).text.strip()
observaçao = soap.find('textarea', {'id': 'observacoes'}).text.strip()

infos.update(
    {
        'Id': soap.find('input', {"name": "id"})['value'],
        'Cep': soap.find('input', {"name": "cep"})['value'],
        'Endereço': soap.find('input', {"name": "endereco"})['value'],
        'Número': soap.find('input', {"name": "numero"})['value'],
        'Complemento': soap.find('input', {"name": "complemento"})['value'],
        'Bairro': soap.find('input', {"name": "bairro"})['value'],
        'Cidade': soap.find('input', {"name": "cidade"})['value'],
        'Uf': soap.find('input', {"name": "uf"})['value'],
        "Dados Bancarios": dados_bancarios,
        "Observações": observaçao 
    })



In [9]:
params = {
    'idAluno': id
}
response_ficha = s.request("GET", URL_FICHA, params=params)

soap = BeautifulSoup(response_ficha.content, 'html.parser')
table_infos = soap.find("div", {"id": "fichaDoAluno"})
ficha = dict()
for linha in table_infos.findAll("div", {"class": "control-group"}):
    ficha.update(
        {
            linha.label.text.strip(): linha.find("div", {"class": "controls"}).text.strip()
        })



In [10]:
URL_FINANCEIRO = URL_BUSCA + 'containerTabClientesAlunosVerMovimentacoes.php'

params = {
    'idAluno': id
}
response_ficha = s.request("GET", URL_FINANCEIRO, params=params)

soap = BeautifulSoup(response_ficha.content, 'html.parser')
financeiro = dict()
nome_coluna = soap.thead.findAll('th')
tds = soap.tbody.findAll('td')
for index, nome in enumerate(nome_coluna[0:-2]):
    financeiro.update(
        {
            nome.text.strip(): tds[index].text.strip()
        }
    )



In [11]:
URL_CREDITO = URL_BUSCA + 'containerTabClientesAlunosVerCreditos.php'

params = {
    'idAluno': id
}
response_credito = s.request("GET", URL_CREDITO, params=params)

soap = BeautifulSoup(response_credito.content, 'html.parser')
creditos = dict()
for tr in soap.tbody.findAll('tr'):
    tds = tr.findAll('td')
    creditos.update(
        {
            "Credito - " + tds[0].text + "/" + tds[1].text: tds[2].text
        }
    )



In [12]:
infos.update(creditos)
infos.update(financeiro)
infos.update(ficha)

In [13]:
infos

{'Id': '4334917',
 'Cep': '24130820',
 'Endereço': 'Travessa Paulistano',
 'Número': '25',
 'Complemento': 'casa02',
 'Bairro': 'Fonseca',
 'Cidade': 'Niterói',
 'Uf': 'RJ',
 'Dados Bancarios': '',
 'Observações': '-',
 'Credito - Exame clínico/Psicotécnico': '0',
 'Credito - Exame clínico/Oftalmológico': '0',
 'Credito - Exame clínico/Toxicológico': '0',
 'Credito - Exame prático/A': '1',
 'Credito - Exame prático/B': '1',
 'Credito - Exame prático/D': '0',
 'Credito - Exame teórico/': '1',
 'Credito - Aulas práticas/A': '16',
 'Credito - Aulas práticas/B': '16',
 'Credito - Aulas práticas/D': '0',
 'Credito - Aulas práticas extras/A': '0',
 'Credito - Aulas práticas extras/B': '0',
 'Credito - Aulas práticas extras/D': '0',
 'Credito - Aulas práticas noturnas/A': '4',
 'Credito - Aulas práticas noturnas/B': '4',
 'Credito - Aulas práticas noturnas/D': '0',
 'Credito - Aulas práticas simulador/A': '0',
 'Credito - Aulas práticas simulador/B': '0',
 'Credito - Aulas práticas simulador/

In [14]:
import json
with open('cfc.json', 'w') as fp:
    json.dump(infos, fp)